## 1. Imports and Setup

We start by importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

## 2. Load Dataset

We load the cleaned resumes dataset from the CSV file.

In [ ]:
df = pd.read_csv('cleaned_resumes.csv')
print(f"Dataset shape: {df.shape}")

## 3. Automatic Feature Selection

We automatically identify the feature columns by excluding the target variable and any non-predictive metadata.

In [ ]:
TARGET = 'experience_level'
ID_COLS = {'name', 'email', 'linkedin', 'github'}

feature_cols = [c for c in df.columns if c != TARGET and c not in ID_COLS]
print(f"Selected Features: {feature_cols}")

for col in feature_cols:
    df[col] = df[col].astype(str).fillna('')

X = df[feature_cols]
y = df[TARGET]

## 4. Data Splitting

We split the data into training (80%) and testing (20%) sets using stratified splitting.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")

## 5. Feature Engineering Pipeline (Weighted)

**Improvements**:
1.  **Feature Weights**: We keep the boosts for 'Total Experience' (5x) and 'Job Title' (3x).
2.  **3-Grams**: We use `ngram_range=(1, 3)` to capture longer phrases (e.g. "senior software engineer"), which helps disambiguate "Mid" roles.

In [ ]:
transformers = []

for col in feature_cols:
    name = f'tfidf_{col}'
    # Restoring (1, 3) n-grams
    transformers.append(
        (name, TfidfVectorizer(ngram_range=(1, 3), min_df=2, max_features=1000), col)
    )

preprocessor = ColumnTransformer(
    transformers=transformers,
    remainder='drop',
)

## 6. Model Training (Random Forest with Class Weights)

**Restored Improvement**: We apply `class_weight={'mid': 2.0, ...}`. This penalizes the model more for getting 'Mid' wrong, encouraging it to focus on correctly classifying the difficult middle class.

In [ ]:
# Restoring class weights to boost Mid performance
class_weights = {'junior': 1.0, 'mid': 3.0, 'senior': 1.0}

rf_model = RandomForestClassifier(
    n_estimators=100, 
    n_jobs=-1,
    random_state=42,
    class_weight=class_weights
)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', rf_model)
])

print(f"Training Random Forest with Class Weights: {class_weights}...")
pipeline.fit(X_train, y_train)
print("Training Complete.")

## 7. Evaluation

We evaluate the Random Forest model performance.

In [ ]:
y_pred = pipeline.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred, labels=['junior', 'mid', 'senior'])
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['junior', 'mid', 'senior'], 
            yticklabels=['junior', 'mid', 'senior'])
plt.title(f'Confusion Matrix: Random Forest (Acc: {acc:.2%})')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()